<h2>Na&iuml;ve Bayes and Feature Extraction</h2>

<p> The Na&iuml;ve Bayes Classifier is a linear classifier based off of bayes rule. 
<br>
The feauture extraction functions take data in the form of words and return data in the form of feature vectors. 
<br>
When data is in feature vector form, this program can perform any binary classification!
<br>
</p>

In [37]:
import numpy as np
import sys
%matplotlib inline

<h3> Naive Bayes</h3>

<p>
An estimate of the class probabilities <b>P(Y)</b>, <b><code>naivebayesPY(x,y)</code></b>
</p>



In [32]:
def naivebayesPY(x,y):
    """
    function [pos,neg] = naivebayesPY(x,y);

    Computation of P(Y)
    Input:
        x : n input vectors of d dimensions (nxd)
        y : n labels (-1 or +1) (nx1)

    Output:
    pos: probability p(y=1)
    neg: probability p(y=-1)
    """
    
    # add one positive and negative example to avoid division by zero ("plus-one smoothing")
    y = np.concatenate([y, [-1,1]])
    n = len(y)
  
    return np.count_nonzero(y==1)/n,np.count_nonzero(y==-1)/n



<p>An estimate of the conditional probabilities <b>P(X|Y)</b>, <b><code>naivebayesPXY</code></b>.
<br>
Using a <b>multinomial</b> distribution as model, this will return the probability vectors  for all features given a class label.
</p> 

In [33]:
#<GRADED>
def naivebayesPXY(x,y):
    """
    function [posprob,negprob] = naivebayesPXY(x,y);
    
    Computation of P(X|Y)
    Input:
        x : n input vectors of d dimensions (nxd)
        y : n labels (-1 or +1) (nx1)
    
    Output:
    posprob: probability vector of p(x|y=1) (1xd)
    negprob: probability vector of p(x|y=-1) (1xd)
    """
    n,d = x.shape
    # add one positive and negative example to avoid division by zero ("plus-one smoothing")
    x = np.concatenate([x, np.ones((2,d))])
    y = np.concatenate([y, [-1,1]])
    
    #return sub array with row vectors where y==1 
    #x_posy = x[np.flatnonzero(y== 1),:]
    #x_negy = x[np.flatnonzero(y==-1),:]
    
    
    feature_counts_posy= np.sum(x[np.flatnonzero(y== 1),:] ,axis=0)
    feature_counts_negy= np.sum(x[np.flatnonzero(y==-1),:] ,axis=0)
    
    return feature_counts_posy/np.sum(feature_counts_posy),feature_counts_negy/np.sum(feature_counts_negy)
    
    




<p>Log ratio, $\log\left(\frac{P(Y=1 | X = xtest)}{P(Y=-1|X= xtest)}\right)$, using Bayes Rule, in <b><code>naivebayes</code></b>.
 <br>
 Result is >1 if Y=1 is the most probable class.
</p>



In [34]:
def naivebayes(x,y,xtest):
    """
    function logratio = naivebayes(x,y);
    
    Computation of log P(Y|X=x1) using Bayes Rule
    Input:
    x : n input vectors of d dimensions (nxd)
    y : n labels (-1 or +1)
    xtest: input vector of d dimensions (1xd)
    
    Output:
    logratio: log (P(Y = 1|X=xtest)/P(Y=-1|X=xtest))
    """
    pof_posy,pof_negy = naivebayesPY(x,y)
    probs_features_posy, probs_features_negy = naivebayesPXY(x,y)
    feature_indices= np.flatnonzero(xtest== 1)
    
    
    return np.log(np.prod(probs_features_posy[feature_indices])*pof_posy/(np.prod(probs_features_negy[feature_indices])*pof_negy) )
    


<p>Naïve Bayes written as a linear classifier in,
<b><code>naivebayesCL</code></b>.

In [35]:
def naivebayesCL(x,y):
    """
    function [w,b]=naivebayesCL(x,y);
    Implementation of a Naive Bayes classifier
    Input:
    x : n input vectors of d dimensions (nxd)
    y : n labels (-1 or +1)

    Output:
    w : weight vector of d dimensions
    b : bias (scalar)
    """
    
    
    
    n, d = x.shape
    ## fill in code here
    pos_prob,neg_prob = naivebayesPXY(x,y)
    py_pos,py_neg   = naivebayesPY(x,y)
    return (np.log(pos_prob)-np.log(neg_prob) ), (np.log(py_pos)-np.log(py_neg))


<p>
<b><code>classifyLinear</code></b>
applies a linear weight vector and bias to a set of input vectors and outputs their predictions. 
 
 

In [36]:
def class_biased(xi,w,b):
    val=np.dot(xi,w)+b
    if val>0:
        return 1
    elif val<0:
        return -1
    else:
        raise ValueError("point lies on hyperplane")
def class_unbiased(xi,w):
    val=np.dot(xi,w)
    if val>0:
        return 1
    elif val<0:
        return -1
    else:
        raise ValueError("point lies on hyperplane")
def classifyLinear(x,w,b=0):
    """
    function preds=classifyLinear(x,w,b)
    
    Make predictions with a linear classifier
    Input:
    x : n input vectors of d dimensions (nxd)
    w : weight vector (dx1)
    b : bias (scalar)
    
    Output:
    preds: predictions (1xn)
    """
    w = w.reshape(-1)
    if b is 0:
        return np.fromiter((class_unbiased(xi,w) for xi in x),int, count=x.shape[0])
    else:
        return np.fromiter((class_biased(xi,w,b) for xi in x),int, count=x.shape[0])


<h3>Feature Extraction</h3>

<p>
<b><code>hashfeatures(word, B, FIX)</code></b> takes a word and considers substrings of the word of length 0-`FIX`, both from he beggining and end of the name as features. A value `1` is put into an array at an index determined by the feature's hashing.  
</p>

In [28]:
def hashfeatures(word, B, FIX):
    v = np.zeros(B)
    for m in range(FIX):
        featurestring = "prefix" + word[:m]
        v[hash(featurestring) % B] = 1
        featurestring = "suffix" + word[-1*m:]
        v[hash(featurestring) % B] = 1
    return v

<p>
<b><code>word2features</code></b> reads every name in the given file and converts it into a B-dimensional feature vector.
</p>

In [29]:
def word2features(filename, B, FIX, LoadFile=True):
    """
    Output:
    X : n feature vectors of dimension B, (nxB)
    """
    # read in words
    if LoadFile:
        with open(filename, 'r') as f:
            words = [x.rstrip() for x in f.readlines() if len(x) > 0]
    else:
        words = filename.split('\n')
    n = len(words)
    X = np.zeros((n, B))
    for i in range(n):
        X[i,:] = hashfeatures(words[i], B, FIX)
    return X



<p><b><code>genTrainFeatures</code></b>, transforms the words into features and loads them into memory. 

In [30]:
def genTrainFeatures(dimension,fix,class1_file,class2_file):
    """
    function [x,y]=genTrainFeatures
    
    This function calls the python script "word2features.py" 
    to convert names into feature vectors and loads in the training data. 
    
    
    Output: 
    x: n feature vectors of dimensionality d [d,n]
    y: n labels {-1 , +1 }
    """
    
    # Load in the data
    X_class1 = name2features(class1_file, B=dimension, FIX=fix)
    X_class2 = name2features(class2_file, B=dimension, FIX=fix)
    X = np.concatenate([X_class1, X_class2])
    
    # Generate Labels
    Y = np.concatenate([-np.ones(len(X_class1)), np.ones(len(X_class2))])
    
    # shuffle data into random order
    ii = np.random.permutation([i for i in range(len(Y))])
    
    return X[ii, :], Y[ii]

<h3> Testing</h3>
You can now test your code with the following interactive name classification script:

In [ ]:
#dimension=130000, fix=6
DIMS = 130000
test_fix=6
class1_file="provide file"
class2_file="provide file"

print('Loading data ...')
X,Y = genTrainFeatures(DIMS,test_fix,class1_file,class2_file)
print('Training classifier ...')
w,b=naivebayesCL(X,Y)
error = np.mean(classifyLinear(X,w,b) != Y)
print('Training error: %.2f%%' % (100 * error))

while True:
    print('Please enter a word>')
    da_word = input()
    if len(da_word) < 1:
        break
    xtest = word2features(da_word,B=DIMS,LoadFile=False)
    pred = classifyLinear(xtest,w,b)[0]
    if pred > 0:
        print("%s, First Class Word.\n" % da_word)
    else:
        print("%s, Second Class Word.\n" % da_word)

<h4>Credits</h4>
  Parts of this webpage markup were copied from or heavily inspired by Killian Weinberg's wonderful <a href="http://www.cs.cornell.edu/courses/cs4780/2018sp/">Machine Learning Course</a>.